<h2 style='color: orange'>Équipe: Louis & Emire</h2>

### 5. Comparaison des performances entre les méthodes linéaires vs méthodes non-linéaires pour la prédiction sur de vraies données transcriptomiques des cancers de TCGA.
#### A. Pour la classification moléculaires entre la régression linéaire et un DNN, quelle méthode fonctionne le mieux et pourquoi?
#### B. Pour la régression entre la régression linéaire et un Auto-Encodeur, quelle méthode fonctionne le mieux et pourquoi?
#### C. Démontrez qu'une PCA 2D est équivalent à la couche interne bottleneck d'Auto-Encodeur linéaire.